In [1]:
#model_checkpoint = "t5-base"
# model_checkpoint = "unicamp-dl/ptt5-base-t5-vocab"
model_checkpoint = 'unicamp-dl/ptt5-base-portuguese-vocab'
#model_checkpoint = "facebook/bart-base"
#model_checkpoint = "unicamp-dl/ptt5-base-t5-vocab"
#model_checkpoint = 'neuralmind/bert-base-portuguese-cased'

In [2]:
PATH_QUESTION = 'finetune_qa_pairs_14k/'

model_name = "QA-ptt5-base-without-retrieval"

In [3]:
import pandas as pd
train_source = pd.read_csv(PATH_QUESTION + 'train.source', sep = "\t", header = None).values.tolist()  

train_target = pd.read_csv(PATH_QUESTION + 'train.target', sep = "\t", header = None).values.tolist()  

test_source = pd.read_csv(PATH_QUESTION + 'test.source', sep = "\t", header = None).values.tolist()  

test_target = pd.read_csv(PATH_QUESTION + 'test.target', sep = "\t", header = None).values.tolist()   

val_source = pd.read_csv(PATH_QUESTION + 'val.source', sep = "\t", header = None).values.tolist()  

val_target = pd.read_csv(PATH_QUESTION + 'val.target', sep = "\t", header = None).values.tolist()  

In [4]:
import pandas as pd
import ast

def preprocess(source, target):
    questions = []
    answer = []
    for i in range(len(source)):
        questions.append( "question: " + source[i][0] )
        answer.append(target[i][0])

    pd_dataset = pd.DataFrame({'question': questions, 'answer' : answer})
    pd_dataset.head()
    
    return(pd_dataset)

pd_dataset_train = preprocess(train_source, train_target)

pd_dataset_val = preprocess(val_source, val_target)

pd_dataset_test = preprocess(test_source, test_target)


In [5]:
from datasets import load_dataset, load_metric, DatasetDict, Dataset

#raw_datasets = load_dataset("xsum")
dataset_train = Dataset.from_pandas(df=pd_dataset_train)
dataset_val = Dataset.from_pandas(df=pd_dataset_val)
dataset_test = Dataset.from_pandas(df=pd_dataset_test)

#dataset.train_test_split()
#train_validtest = dataset.train_test_split(0.1)
#valid_test = train_validtest['test'].train_test_split(0.5)

raw_datasets = DatasetDict({
    'train': dataset_train,
    'validation': dataset_val,
    'test': dataset_test})
metric = load_metric("rouge")



In [6]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [7]:
from transformers import AutoTokenizer, T5Tokenizer, T5TokenizerFast, TFT5Model, TFT5ForConditionalGeneration

#tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [8]:
max_input_length = 512
max_target_length = 32

def preprocess_input(examples):
    model_inputs = tokenizer(examples["question"], max_length=max_input_length, truncation=True)
    return model_inputs

def preprocess_function(examples):
    model_inputs = preprocess_input(examples)
    # Setup the tokenizer for targets
    labels = tokenizer(examples["answer"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [9]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

In [10]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [11]:
batch_size = 4
args = Seq2SeqTrainingArguments(
    model_name,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=30,
    predict_with_generate=True,
    fp16=True,
    gradient_accumulation_steps = 4
)

In [12]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [13]:
import nltk
#nltk.download('punkt')
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [14]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/home/blab-answerer/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,No log,2.120113,19.624100,4.091200,19.620100,19.562700,4.252700
1,2.757300,1.952777,23.009800,5.486300,23.013800,22.941400,4.361100
2,2.757300,1.869116,24.206800,6.165700,24.166900,24.143200,4.439500
3,2.017600,1.806282,25.359700,6.510400,25.247000,25.187100,4.460400


/home/blab-answerer/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/blab-answerer/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/blab-answerer/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [ ]:
a = trainer.predict(tokenized_datasets['test'], max_length=max_target_length)

In [ ]:
test_outputs = []
test_labels = []
for i in range(len(a.predictions)):
    test_outputs.append(tokenizer.decode(a.predictions[i]))
    test_labels.append(raw_datasets['test'][i]['answer'])
pd_output = pd.DataFrame({'predicted' : test_outputs, 'label' : test_labels})

In [ ]:
pd_output.head()

In [ ]:
raw_datasets['test'][0]['question']